In [1]:
import sys
sys.path.append("..")
sys.path.append("../src")

import config
from dataIO import load_data
from vectorDB import QdrantManager
from vectorDB import PaperEmbed

In [ ]:
import sys
sys.path.append("..")
sys.path.append("../src")

import threading, time, requests, json
import uvicorn
from services.search.api import app
from services.search.workers import start_search_engine

In [ ]:
def run_api():
    uvicorn.run(app, host="127.0.0.1", port=2000, log_level="info")

api_thread = threading.Thread(target=run_api, daemon=True)
api_thread.start()

time.sleep(2)

INFO:     Started server process [17257]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:2000 (Press CTRL+C to quit)


INFO:     127.0.0.1:59730 - "POST /search HTTP/1.1" 200 OK
INFO:     127.0.0.1:59746 - "POST /search HTTP/1.1" 200 OK
INFO:     127.0.0.1:52974 - "GET /user/jul/notebooks/test-notebook HTTP/1.1" 200 OK
INFO:     127.0.0.1:52990 - "GET /user/alter/notebooks/another-notebook HTTP/1.1" 200 OK
INFO:     127.0.0.1:35764 - "POST /search HTTP/1.1" 200 OK
INFO:     127.0.0.1:35776 - "POST /search HTTP/1.1" 200 OK


In [ ]:
t1 = start_search_engine()
print("Search worker started!")

Search worker started!


INFO:vectorDB.rerank:Will load reranker-default with status production
INFO:vectorDB.rerank:Loading reranker-default - version-1
INFO:vectorDB.rerank:models:/reranker-default/1 loaded
INFO:services.search.search:Saved results for 5c022ffb48a537cbf1e0/04f84e7967623db1aa8f
INFO:services.search.search:Saved results for 7c0ea37f8baaead4a586/989a7f31f54cd8825be9
INFO:services.search.search:Saved results for 5c022ffb48a537cbf1e0/04f84e7967623db1aa8f
INFO:services.search.search:Saved results for 7c0ea37f8baaead4a586/989a7f31f54cd8825be9


In [4]:
import requests

API_URL = "http://localhost:2000"
user_id = "jul"
notebook_id = "test-notebook"
query = "what is the role of the amygdala?"

In [9]:
payload1 = {
    "user_id": user_id,
    "notebook_id": notebook_id,
    "query": query
}
payload2 = {
    "user_id": "alter",
    "notebook_id": "another-notebook",
    "query": "what are graph neural networks?"
}
res = requests.post(f"{API_URL}/search", json=payload1)
res.json()
res = requests.post(f"{API_URL}/search", json=payload2)
res.json()
# res = requests.post(f"{API_URL}/search", json=payload1)
# res.json()
# res = requests.post(f"{API_URL}/search", json=payload2)
# res.json()

{'task_id': '7c0ea37f8baaead4a586/989a7f31f54cd8825be9'}

In [6]:
res = requests.get("http://localhost:2000/user/jul/notebooks/test-notebook").json()
res = requests.get("http://localhost:2000/user/alter/notebooks/another-notebook").json()

In [8]:
[(r['qdrant_score'], r['user_score']) for r in res['result']['result']]

[(0.89264095, None),
 (0.88703305, None),
 (0.889835, None),
 (0.88846755, None),
 (0.88784057, None),
 (0.8844021, None),
 (0.8760797, None),
 (0.8737221, None),
 (0.8734746, None),
 (0.8731606, None),
 (0.87216496, None),
 (0.87349945, None),
 (0.8717787, None),
 (0.8724871, None),
 (0.8702082, None),
 (0.87014115, None),
 (0.86749816, None),
 (0.8661747, None),
 (0.864318, None),
 (0.8642633, None),
 (0.8644265, None),
 (0.8627149, None),
 (0.861989, None),
 (0.8608699, None),
 (0.8580732, None),
 (0.8591125, None),
 (0.85805976, None),
 (0.8617748, None),
 (0.85515964, None),
 (0.8552043, None),
 (0.8560201, None),
 (0.85394645, None),
 (0.8516691, None),
 (0.8509387, None),
 (0.8514811, None),
 (0.8503893, None),
 (0.8487557, None),
 (0.84755576, None),
 (0.8474346, None),
 (0.8468143, None),
 (0.8442084, None),
 (0.84367245, None),
 (0.84418845, None),
 (0.84405714, None),
 (0.8426778, None),
 (0.844103, None),
 (0.8422974, None),
 (0.8420732, None),
 (0.8420416, None),
 (0.84100

In [6]:
import time

task_id = res.json()["task_id"]

for _ in range(10):
    time.sleep(2)  # wait for worker/consumer to process
    result = requests.get(f"{API_URL}/search/result/{task_id}")
    if result.status_code == 200:
        break

result.json()

{'task_id': '5c022ffb48a537cbf1e0b4e4148f00429a13984326d5b8d59ebd0e98d3e446fd/04f84e7967623db1aa8f1baa8807ce44556e5b9cb3daad7c009dc50bbb22395d',
 'result': {'id': '5c022ffb48a537cbf1e0b4e4148f00429a13984326d5b8d59ebd0e98d3e446fd/04f84e7967623db1aa8f1baa8807ce44556e5b9cb3daad7c009dc50bbb22395d',
  'task_id': '5c022ffb48a537cbf1e0b4e4148f00429a13984326d5b8d59ebd0e98d3e446fd/04f84e7967623db1aa8f1baa8807ce44556e5b9cb3daad7c009dc50bbb22395d',
  'query': 'what is the role of the amygdala?',
  'user_id': 'jul',
  'notebook_id': 'test-notebook',
  'result': [{'paperIdQ': '2a014b44-f60f-5963-999e-b5ab3a7423d0',
    'paperId': '8f213bf668b4bce273b7e0e000da41ff790bb1a8',
    'citationCount': 0,
    'abstract': 'None',
    'publicationYear': 2025,
    'influentialCitationCount': 0.0,
    'openAccessPdf': 'https://www.researchsquare.com/article/rs-3939709/latest.pdf',
    'authorName': 'Yingning Zhang,Sujuan Shi,E. Mao,Yuling Chen,Jing Chen,Miao Tian,Fensheng Huang,Zhiping Cai,Yunqing Li,Zhenzhen K

In [32]:
def run_api():
    uvicorn.run(app, host="127.0.0.1", port=2000, log_level="info")

api_thread = threading.Thread(target=run_api, daemon=True)
api_thread.start()

time.sleep(2)

INFO:     Started server process [87659]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('127.0.0.1', 2000): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


In [33]:
r = requests.post("http://localhost:2000/search",
                  json={"query": "what is a species?"})
task_id = r.json()["task_id"]
print("task_id:", task_id)

r = requests.post("http://localhost:2000/search",
                  json={"query": "what is the amygdala?"})
task_id = r.json()["task_id"]
print("task_id:", task_id)

task_id: 80c18e21-4561-4c02-a0da-4bc7d2f5633b
task_id: 8ab71f0d-ac47-4ac9-932f-eed0c9a294ac


%6|1747476235.036|FAIL|rdkafka#consumer-13| [thrd:GroupCoordinator]: GroupCoordinator: localhost:29092: Disconnected (after 4093206ms in state UP)
%6|1747476235.036|FAIL|rdkafka#consumer-4| [thrd:GroupCoordinator]: GroupCoordinator: localhost:29092: Disconnected (after 17114342ms in state UP)
%6|1747476235.036|FAIL|rdkafka#consumer-11| [thrd:localhost:29092/1]: localhost:29092/1: Disconnected (after 16429358ms in state UP)
%6|1747476235.036|FAIL|rdkafka#consumer-15| [thrd:GroupCoordinator]: GroupCoordinator: localhost:29092: Disconnected (after 4006808ms in state UP)
%6|1747476235.036|FAIL|rdkafka#consumer-4| [thrd:localhost:29092/1]: localhost:29092/1: Disconnected (after 17114341ms in state UP)
%6|1747476235.036|FAIL|rdkafka#consumer-10| [thrd:localhost:29092/1]: localhost:29092/1: Disconnected (after 16606068ms in state UP)
%6|1747476235.037|FAIL|rdkafka#consumer-14| [thrd:GroupCoordinator]: GroupCoordinator: localhost:29092: Disconnected (after 4063076ms in state UP)
%6|1747476235.

In [30]:
import time
time_out = 10
start_time = time.time()
with requests.get(f"http://localhost:2000/search/stream/{task_id}",
                  stream=True) as resp:
    if time.time() - start_time < time_out:
        for line in resp.iter_lines():
            if not line or line.startswith(b":"): 
                continue
            start_time = time.time()
            event = json.loads(line.decode().lstrip("data: "))
            print(event)

{'id': '26a0efe9-9acc-45bf-80e6-03408a9f3319', 'result': [{'paperIdQ': '2a014b44-f60f-5963-999e-b5ab3a7423d0', 'paperId': '8f213bf668b4bce273b7e0e000da41ff790bb1a8', 'citationCount': 0, 'abstract': 'None', 'publicationYear': 2025, 'influentialCitationCount': 0.0, 'openAccessPdf': 'https://www.researchsquare.com/article/rs-3939709/latest.pdf', 'authorName': 'Yingning Zhang,Sujuan Shi,E. Mao,Yuling Chen,Jing Chen,Miao Tian,Fensheng Huang,Zhiping Cai,Yunqing Li,Zhenzhen Kou', 'venue': 'Brain Research', 'main_text': [], 'title': 'Tac1-expressing neurons in the central amygdala predominantly mediate histamine-induced itch by receiving inputs from parabrachial Tac1-expressing neurons', 'publicationDate': '2025-02-01', 'fieldsOfStudy': 'Medicine', 'score_qdrant': 0.8348143}, {'paperIdQ': 'ee7ecd4b-4d70-5827-b8e1-16dbbbd4ac0a', 'paperId': '481923e7e529525e155481bb0bd78cc978c124ac', 'citationCount': 0, 'abstract': 'None', 'publicationYear': 2025, 'influentialCitationCount': 0.0, 'openAccessPdf'

KeyboardInterrupt: 

In [2]:
embedEngine = PaperEmbed(model_name=config.MODEL_EMBED)
qdrant = QdrantManager()

In [3]:
query = "what defines a species?"

In [4]:
query_emb = embedEngine.embed_queries([query])

In [5]:
len(query_emb[0])

1024

In [6]:
res = await qdrant.client.get_collection(qdrant.collection_name_ids)
print(res.config.params.vectors.size)

1024


In [7]:
res = await qdrant.query_batch_streamed(query_vectors=[query_emb.tolist()], topK=50, filter_payloads=[['publicationDate', 'gt', '2025-01-01'],['paperId','==',['0027ea27f5986f9c6ee9776e10289c7f2aa089ca','a1ea5b916228bd88b391567eb2d57a5bf02dc6ff']]], with_vectors=True, collection_name=qdrant.collection_name_ids)

In [ ]:
res[0].points[0].payload | {"score_qdrant": res[0].points[0].score}

{'paperIdQ': '6465b1c5-8000-51d5-9669-96fb75109ffd',
 'paperId': '0027ea27f5986f9c6ee9776e10289c7f2aa089ca',
 'citationCount': 1,
 'abstract': 'None',
 'publicationYear': 2025,
 'influentialCitationCount': 0.0,
 'openAccessPdf': 'gs://arxiv-dataset/arxiv/arxiv/pdf/2501/2501.13553v1.pdf',
 'authorName': 'Robert Szafarczyk,S. Nabi,W. Vanderbauwhede',
 'venue': 'International Conference on Compiler Construction',
 'main_text': 'Compiler Support for Speculation in Decoupled Access/Execute Architectures Syed Waqar Nabi University of Glasgow Glasgow, United Kingdom syed.nabi@glasgow.ac.uk Robert Szafarczyk University of Glasgow Glasgow, United Kingdom robert.szafarczyk@glasgow.ac.uk Wim Vanderbauwhede University of Glasgow Glasgow, United Kingdom wim.vanderbauwhede@glasgow.ac.uk Abstract Irregular codes are bottlenecked by memory and communi- cation latency. Decoupled access/execute (DAE) is a common technique to tackle this problem. It relies on the compiler to separate memory address gener

In [8]:
res = await qdrant.query_batch_streamed(query_vectors=query_emb.tolist(), topK=50, offset=50, with_vectors=True, collection_name=qdrant.collection_name_ids)
res[0].points[0]

ScoredPoint(id='d0aa7e18-5e0c-57ad-ab2e-5c4b355b5b3e', version=4337, score=0.79184455, payload={'paperIdQ': 'd0aa7e18-5e0c-57ad-ab2e-5c4b355b5b3e', 'paperId': '82696e684a577ca208c782b39c1285b53643a26a', 'citationCount': 0, 'abstract': 'None', 'publicationYear': 2025, 'influentialCitationCount': 0.0, 'openAccessPdf': 'https://figshare.com/articles/journal_contribution/Importance_of_the_integrated_use_of_bioacoustics_and_mist_nets_for_bat_surveys/28450887/1/files/52498398.pdf', 'authorName': 'Jeanneson Silva de Sales,Hannah Larissa Nunes de Lima,F. L. Rocha,P. Cordeiro-Estrela,Patrício Adriano da Rocha', 'venue': 'Bioacoustics', 'main_text': 'Supplementary  Figure  1.  Accumulation  curves  for  bat  species,  based  on  data  from acoustic  monitoring  of  five  forest  fragments,  from  the  Metropolitan  Region  of  João Pessoa/PB,  northeastern  Brazil.  a)  Overall  Forest  Fragments;  b)  Floresta  Nacional  da Restinga de Cabedelo; c) Jardim Florestal Municipal da Cidade Verde; d)

In [ ]:
res = await qdrant.query_batch_streamed(query_vectors=[query_emb.tolist()], topK=100, filter_payloads=[['publicationDate', 'gt', '2025-01-01']], with_vectors=True, collection_name=qdrant.collection_name_ids)
print(res[0].groups[1].hits[0].payload['title'])
res = await qdrant.query_batch_streamed(query_vectors=[res[0].groups[1].hits[0].vector], topK=100, offset=100, filter_payloads=[['publicationDate', 'gt', '2025-01-01']], with_vectors=True, collection_name=qdrant.collection_name_ids)

In [ ]:
res

In [ ]:
match_titles = [res[0].groups[k].lookup.payload['title'] for k in range(len(res[0].groups))]
[print(title) for title in match_titles];

In [28]:
match_titles = [res[0].groups[k].lookup.payload['title'] for k in range(len(res[0].groups))]
[print(title) for title in match_titles];

Land use system effects on biodiversity of litter ants (Hymenoptera: Formicidae) in tropical rainforest, South Region of Cameroon
Species Delimitation and Cryptic Diversity in Rheotanytarsus Thienemann & Bause, 1913 (Diptera: Chironomidae) Based on DNA Barcoding
DNA barcoding reveals mislabeling of endangered sharks sold as swordfish in New England fish markets
Taxonomic implications of leaf anatomy in Thai Smilax (Smilacaceae)
Disentangling the Taxonomy of Parides burchellanus (Lepidoptera: Papilionidae) and Related Taxa.
TaxDiff: taxonomic-guided diffusion model for protein sequence generation
Where are my nematodes? labelling and visualising entomopathogenic nematodes in vivo using carbon quantum dots.
FACdb: a comprehensive resource for genes, gut microbiota, and metabolites in farm animals
Adaptive Divergence and Functional Convergence: The Evolution of Pulmonary Gene Expression in Amphibians of the Qingzang Plateau.
The birds of the archaeological sites of the Acropolis area (Ath

In [29]:
import nest_asyncio
nest_asyncio.apply()

ref_ids = []
for k in range(6, 4, -1):
    if not ref_ids:
        res = await qdrant.query_batch_streamed(query_vectors=query_emb[0][k:k+1].tolist(), vector_name=f"embed_{k}", topK=50, filter_payloads=[['publicationDate', 'gt', '2025-02-01']])
    else:
        res = await qdrant.query_batch_streamed(query_vectors=query_emb[0][k:k+1].tolist(), vector_name=f"embed_{k}", topK=50, filter_payloads=[['paperId', 'in', ref_ids]])
    ref_ids = [res[0].groups[k].lookup.payload['paperId'] for k in range(len(res[0].groups))]
    match_titles = [res[0].groups[k].lookup.payload['title'] for k in range(len(res[0].groups))]
    match_texts = [res[0].groups[k].lookup.payload['main_text'] for k in range(len(res[0].groups))]
    if not ref_ids:
        ref_titles = match_titles
    # print(" ")

[print(title) for title in match_titles];

Where are my nematodes? labelling and visualising entomopathogenic nematodes in vivo using carbon quantum dots.
Taxonomic implications of leaf anatomy in Thai Smilax (Smilacaceae)
The Spiky Side of Urban Wildlife, First Detection of a Zoonotic Assemblage of Giardia duodenalis in European Hedgehogs (Erinaceus europaeus) from Italy
Dscamb regulates cone mosaic formation in zebrafish via filopodium-mediated homotypic recognition
Land use system effects on biodiversity of litter ants (Hymenoptera: Formicidae) in tropical rainforest, South Region of Cameroon
Erratum: DAVID L. EMERY, NATHAN J. EMERY & LINDSAY W. POPPLE (2025) Description of three new cicada species in the genus Yoyetta Moulds (Hemiptera: Cicadidae: Cicadettinae), from eastern Australia. Zootaxa, 5590 (2): 151-184.
Importance of the integrated use of bioacoustics and mist nets for bat surveys
Two new species of Crassitegula (Sebdeniaceae, Sebdeniales, Rhodophyta) extend the diversity and biogeographic range of the genus to th

In [72]:
res[0].groups[k].lookup.payload.keys()

dict_keys(['paperIdQ', 'paperId', 'publicationDate', 'publicationYear', 'citationCount', 'venue', 'fieldsOfStudy', 'openAccessPdf', 'abstract', 'title', 'influentialCitationCount', 'authorName', 'main_text'])

In [31]:
[print(title) for title in match_titles];

Land use system effects on biodiversity of litter ants (Hymenoptera: Formicidae) in tropical rainforest, South Region of Cameroon
Species Delimitation and Cryptic Diversity in Rheotanytarsus Thienemann & Bause, 1913 (Diptera: Chironomidae) Based on DNA Barcoding
Sigmodontine rodent diversity: The Frankenstein paradox
Comparison of growth, relative abundance, and diet of three sympatric Hemiandrus ground wētā (Orthoptera, Anostostomatidae) in a New Zealand Forest
Species and site contributions to Odonata adult assemblage uniqueness in Brazilian Amazon streams
DNA barcoding reveals mislabeling of endangered sharks sold as swordfish in New England fish markets
Taxonomic implications of leaf anatomy in Thai Smilax (Smilacaceae)
TAXONOMY OF THE KATYDIDS (ORTHOPTERA: TETTIGONIIDAE) FROM EAST ASIA AND ADJACENT ISLANDS. COMMUNICATION 17
Disentangling the Taxonomy of Parides burchellanus (Lepidoptera: Papilionidae) and Related Taxa.
From virtue to sin: Is the installation of bat boxes an effect

In [7]:
from transformers import pipeline
import torch

# model_id = "unsloth/Mistral-Small-24B-Instruct-2501-unsloth-bnb-4bit"
model_id = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"
pipe = pipeline(
    "text-generation",
    model=model_id)

`low_cpu_mem_usage` was None, now default to True since model is quantized.
Device set to use cuda:0


In [10]:
articles = '\n\nabstract#'.join(f"{k}\n" + txt for k, txt in enumerate(match_texts[2:3]))
query="What is the role of amygdala?"
messages = [
    {"role": "system", "content": """
Summarize the article below to help answer a question. 
Do not directly answer the question, instead summarize the evidence present in the excerpt that
help answer the question. 
Reply ’Not applicable’ if the excerpt is irrelevant. 
At the end of your response, provide a score from 1-10 on a newline indicating the relevance
of the text to the question. 
DO NOT EXPLAIN YOUR SCORE.
     """},
    {"role": "user", "content": f"""
Question: {query}
Article: 
{articles}

Relevant Information Summary:"""}
]
# query="What is the role of amygdala?"
# messages = f"""<s>[SYSTEM_PROMPT]Summarize the article below to help answer a question. 
# Do not directly answer the question, instead summarize the evidence present in the excerpt that
# help answer the question. 
# Reply ’Not applicable’ if the excerpt is irrelevant. 
# At the end of your response, provide a score from 1-10 on a newline indicating the relevance
# of the text to the question. 
# Do not explain your score.[/SYSTEM_PROMPT]

# [INST]
# Question: {query}
# Article: 
# {articles}

# Relevant Information Summary:[/INST]"""

outputs = pipe(
    messages,
    max_new_tokens=10000,
)
# print(outputs[0]["generated_text"][-1]['content'])

In [11]:
print(outputs[0]["generated_text"][-1]['content'])

 The article discusses the role of dynorphin-expressing neurons in the central amygdala (CeADyn) in alcohol drinking. The study used fiber photometry to record real-time activity of these neurons in male and female mice during voluntary alcohol consumption, water consumption, and sucrose consumption. The results showed that there is a unique functional signature for alcohol in the CeADyn neuron population, with a large increase in calcium transients after bouts of licking for alcohol, and only modest increases during licking for water or 0.5% sucrose. This increase in activity was present even when controlling for differences in drinking behavior unique to alcohol, such as longer bout durations. The study suggests that understanding the specific brain mechanisms underlying the behavioral and physiological responses to alcohol that promote alcohol misuse is important for understanding the role of these neurons in alcohol drinking.

Relevance Score: 10

Explanation: The article provides 

In [37]:
print(outputs[0]["generated_text"].split("[/INST]")[-1])

The article discusses a study that investigates the activity of dynorphin-expressing neurons in the central amygdala (CeADyn) during alcohol drinking in mice. Key findings include:

- CeADyn neurons show a large increase in calcium transients after bouts of licking for alcohol, compared to modest increases for water or sucrose.
- The unique engagement of CeADyn neurons during alcohol consumption is not fully explained by differences in drinking behavior, such as longer bout durations for alcohol.
- The study used fiber photometry to record neuronal activity and multilevel modeling to analyze the data, revealing significant effects of time and solution type on neuronal activity.
- The study also explored the relationship between drinking characteristics (such as bout duration and temporal occurrence) and neuronal activity, finding that while these characteristics modulate signal, they do not fully explain the unique alcohol-evoked CeADyn activity.
- The study suggests that the unique ph

In [19]:
print(messages)

<s>[SYSTEM_PROMPT]You are a research assistant. Answer the user’s query as accurately as possible using your knowledge and the provided articles.     Do not summarize the abstracts unless explicitly asked. Use your knowledge first if the question is general. Try as much as possible to cite the provided abstract which are      referenced as abstract#1, abstract#2, etc[/SYSTEM_PROMPT]     [INST]Here are the abstracts of recent papers:

0
AmygdalaGo-BOLT3D: A boundary learning transformer for tracing human amygdala 1Center for Brain Imaging Science and Technology, Zhejiang University, Hangzhou, 310027, Zhejiang, China. 2College of Biomedical Engineering and Instrument Science, Zhejiang University, Hangzhou, 310027, China. 3State Key Laboratory of Cognitive Neuroscience and Learning, Beijing Normal University, Beijing, 100875, China. 4Faculty of Psychology, Beijing Normal University, Beijing, 100875, Beijing, China. 5School of Psychology, Capital Normal University, Beijing, 100190, China. 